# Trade Algorithm to Automate Short Position Trades Using Daily Patterns


Only allowed short positions (selling first, buying later)


Max. stop loss 7% of initial investment


Max. take profit 14%-21% of initial investment (2:1/3:1 risk-reward ratio depending on leverage)


Support and Resistance levels calculated using anticipated pct. change in price over last 5 days


# Key Considerations Added to Simplified Structure:
## 1. Get historical data of the asset(s)
1.1. Verify data quality and check for missing values/outliers.


1.2. Account for trading volumes, volatility, and any specific asset-related adjustments.


1.3. Pull leverage-related information.


## 2. Calculate entry and exit points
2.1. Calculate anticipated pct. change in the price over the last 5 trading days.


2.2. Calculate support and resistance levels.


2.3. Ensure support/resistance levels align with leverage and volatility.
## 3. Risk Management
3.1. Set stop loss (max 7%) and take profit (between 14%-21%) based on risk-reward ratio.


3.2. Calculate position size that adheres to max stop loss and keeps total risk within acceptable boundaries.


3.3. Take leverage into consideration when calculating actual exposure and risk.
## 4. Execute the trade using Kraken REST API
4.1. Set up connection with Kraken API (handle authentication).


4.2. Monitor trade after execution for any trailing stop conditions or adjustments.


4.3. Log trades, including entries, exits, wins, and losses for further analysis. -->

In [22]:
# Import necessary libraries
import numpy as np
import pandas as pd
import requests
import datetime
import hashlib
from difflib import get_close_matches
import hmac
import base64
import time
import tensorflow as tf
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import urllib.parse

# Import config file
import sys
sys.path.insert(0, '../')
import config as cfg

In [13]:
api_key = cfg.api_key
private_key = cfg.api_priv

## API Fetchin

In [133]:
api_url = "https://api.kraken.com"

# Create dictionary to map API endpoints to their respective names and respective request types
api_endpoints = {
    'Assets': '/0/public/Assets',
    'AssetPairs': '/0/public/AssetPairs',
    'Ticker': '/0/public/Ticker',
    'OHLC': '/0/public/OHLC',
    'default OHLC': '/0/public/OHLC',

    'Balance': '/0/private/Balance',
    'ExtendedBalance': '/0/private/BalanceEx',
    'Ledgers': '/0/private/Ledgers',
    'QueryLedgers': '/0/private/QueryLedgers',
    'TradeVolume': '/0/private/TradeVolume',
    'TradesHistory': '/0/private/TradesHistory',
    'OpenOrders': '/0/private/OpenOrders',

}

# Function to generate a nonce
def generate_nonce():
    nonce = str(int(1000 * time.time()))
    return nonce

# Function to get Kraken signature
def get_kraken_signature(urlpath, data, secret):
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode() 

# Function to make Kraken API request
def kraken_request(uri_path, data, api_key, api_sec, headers=None):
    if headers is None:
        headers = {}

    headers['API-Key'] = api_key
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

#Function to make more non-trivial Kraken API get request
def kraken_get_request(uri_path, data=None, headers=None):
    if uri_path != api_endpoints['OHLC']:
        if headers is None:
            headers = {
                'Accept': 'application/json'
            }
        
        headers.update(headers)
        req = requests.get((api_url + uri_path), headers=headers, data=data)
    elif uri_path == api_endpoints['OHLC']:
        temp_endpoint = api_endpoints['OHLC'] + '?pair=' + data['pair'] + '&interval=' + data['interval']
        req = requests.get((api_url + temp_endpoint), headers=headers)
    return req


In [134]:
# Possible intervals: 1, 5, 15, 30, 60, 240, 1440, 10080, 21600 in minutes i.e., 1 minute, 5 minutes, 15 minutes, 30 minutes, 1 hour, 4 hours, 1 day, 1 week, 1 month
# Possible tenures: 1D (1440), 7D (10080), 1M (43200), 3M (129600), 6M (259200), 1Y (518400) - corresponding intervals are tenure/720 to maximize data points from a single request
possible_intervals =[1, 5, 15, 30, 60, 240, 1440, 10080, 21600]
possible_timeframes = {'1D': 1440, '7D': 10080, '1M': 43200, '3M': 129600, '6M': 259200, '1Y': 518400}

tenure = '1D'


In [135]:
# 1. Get Historical OHLC Data of the Asset(s) using Kraken REST API
# Function to grab the OHLC data for a given list of asset pairs
def grab_ohlc_data(assetPairs,tenure):
    # divide timerframe by 720 to get the interval but use the next larger closet possible interval
    interval = min([i for i in possible_intervals if i >= possible_timeframes[tenure]/720], default=possible_intervals[-1])
    interval = str(interval)
    # Construct since parameter for the OHLC request using tenure and datetime unix converted timestamp, i.e., subtracting the tenure from the current time and equating it to the since parameter
    since = int(time.time()) - possible_timeframes[tenure]*60
    since = str(since)

    # Construct the Kraken API request and get the OHLC data for the given asset pairs, ohlc grabbing requires use of a temporary endpoint for the OHLC url
    ohlcDict = {}
    for assetPair in assetPairs:
        resp = kraken_get_request(api_endpoints['OHLC'], {"pair": assetPair, "interval": interval, "since": since}).json()
        if resp['error'] == KeyError:
            #skip this asset pair
            continue
        ohlcDict[assetPair] = list(resp['result'].values())
        # To process the response, we need to extract the OHLC data from the response particularly the tick data array and the last timestamp
    # Append the OHLC data to a dataframe and return the dataframe with columns: Time, Open, High, Low, Close, Volume, Count, name it after the asset pair
    return ohlcDict

# 2. Calculate Entry and Exit Points
# 2.1 Calculate anticipated percentage change in price using Linear Regression or Neural Networks
def calculate_anticipated_pct_change(data):
    # Using Linear Regression to Predict Percentage Change
    model = LinearRegression()
    data['Days'] = range(len(data))  # Add a column representing the number of days for fitting
    X = data[['Days']]
    y = data['Close']
    model.fit(X, y)
    predicted_price = model.predict([[len(data)]])[0]  # Predicting the next value
    anticipated_pct_change = ((predicted_price - data['Close'].iloc[-1]) / data['Close'].iloc[-1]) * 100
    
    # Alternatively, using a simple Neural Network with TensorFlow
    # Prepare data for Neural Network
    features = np.array(data[['Days']])
    target = np.array(data['Close'])
    model_nn = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu', input_shape=(1,)),
        tf.keras.layers.Dense(1)
    ])
    model_nn.compile(optimizer='adam', loss='mse')
    model_nn.fit(features, target, epochs=100, verbose=0)
    predicted_price_nn = model_nn.predict([[len(data)]])[0][0]
    anticipated_pct_change_nn = ((predicted_price_nn - data['Close'].iloc[-1]) / data['Close'].iloc[-1]) * 100

    # Combining the results from Linear Regression and Neural Network for better estimation
    anticipated_pct_change = (anticipated_pct_change + anticipated_pct_change_nn) / 2
    return anticipated_pct_change

# 2.2 Calculate support and resistance levels
def calculate_support_resistance(data, anticipated_pct_change):
    current_price = data['Close'].iloc[-1]
    resistance_level = current_price * (1 + anticipated_pct_change / 100)
    support_level = current_price * (1 - anticipated_pct_change / 100)
    return support_level, resistance_level

# 3. Risk Management
# 3.1 Calculate stop loss and take profit levels based on desired risk-reward ratio
def calculate_risk_management_levels(entry_price, risk_reward_ratio):
    stop_loss = entry_price * (1 - 0.07)  # Max stop loss 7%
    take_profit = entry_price * (1 + (risk_reward_ratio * 0.07))  # Take profit based on risk-reward ratio
    return stop_loss, take_profit

# 3.2 Calculate position size
def calculate_position_size(total_capital, entry_price, stop_loss):
    risk_per_trade = total_capital * 0.02  # Assume 2% risk of total capital per trade
    position_size = risk_per_trade / abs(entry_price - stop_loss)
    return position_size

# 4. Execute Trade Using Kraken REST API
def execute_trade(api_key, private_key, asset_symbol, position_size, entry_price, order_type="sell"):
    api_url = "https://api.kraken.com/0/private/AddOrder"
    nonce = str(int(time.time() * 1000))
    
    # Data payload for the trade
    data = {
        "nonce": nonce,
        "ordertype": "limit",
        "type": order_type,  # "sell" for short positions
        "volume": position_size,
        "pair": asset_symbol,
        "price": entry_price,
        "leverage": "2:1"  # Example leverage, can be adjusted
    }
    
    # Create the Kraken API signature
    postdata = urllib.parse.urlencode(data)
    message = (nonce + postdata).encode()
    sha256 = hashlib.sha256(nonce.encode() + postdata.encode()).digest()
    hmac_key = base64.b64decode(private_key)
    signature = hmac.new(hmac_key, b"/0/private/AddOrder" + sha256, hashlib.sha512)
    signature_b64 = base64.b64encode(signature.digest())
    
    headers = {
        "API-Key": api_key,
        "API-Sign": signature_b64.decode()
    }
    
    response = requests.post(api_url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to execute trade: " + response.text)


In [136]:
assetList = ['XXRP','SOL', 'XXBT']

In [139]:
allAssetPairs = kraken_get_request(api_endpoints['AssetPairs'], headers={'country_code':'GB'}).json()['result']

# Filter the asset pairs based on the assetList and quote currency
# tradeablePairs = [
#     {key: asset[key] for key in ['altname','base', 'quote', 'ordermin', 'fees', 'fees_maker','leverage_buy', 'leverage_sell']}
#     for asset in allAssetPairs.values() if asset['quote'] == 'ZUSD'
# ]

tradeablePairs = [
    {key: asset[key] for key in ['altname','base', 'quote', 'ordermin', 'fees', 'fees_maker','leverage_buy', 'leverage_sell']}
    for asset in allAssetPairs.values() if asset['wsname'][-3:] == 'USD'
]


# Check if asset['base'] is in the assetList
filteredPairs = [asset for asset in tradeablePairs if asset['base'] in assetList]


# Get the OHLC data for the filtered asset pairs
#ohlcData = grab_ohlc_data([asset['base'] + asset['quote'] for asset in filteredPairs], tenure)
ohlcData = grab_ohlc_data([asset['altname'] for asset in filteredPairs], tenure)
print(ohlcData)

# Convert to DataFrame for each asset pair, then add 9 vs 21 day rolling averages
ohlcDFs = {}
for assetPair in ohlcData.keys():
    ohlcDFs[assetPair] = pd.DataFrame(ohlcData[assetPair][:], columns=['Time', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Volume', 'Count'])
    ohlcDFs[assetPair]['Time'] = pd.to_datetime(ohlcDFs[assetPair]['Time'], unit='s')
    ohlcDFs[assetPair].set_index('Time', inplace=True)
    ohlcDFs[assetPair]['9_day_MA'] = ohlcDFs[assetPair]['Close'].rolling(window=9).mean()
    ohlcDFs[assetPair]['21_day_MA'] = ohlcDFs[assetPair]['Close'].rolling(window=21).mean()



{'SOLUSD': [[[1733746500, '226.70', '226.93', '226.29', '226.93', '226.75', '470.80047674', 67], [1733746800, '226.95', '227.41', '226.95', '227.41', '227.31', '206.79896980', 61], [1733747100, '227.35', '227.50', '227.11', '227.50', '227.35', '124.55780621', 41], [1733747400, '227.50', '227.61', '227.00', '227.00', '227.24', '72.11000061', 49], [1733747700, '226.99', '226.99', '226.51', '226.69', '226.78', '72.19434124', 37], [1733748000, '226.68', '226.92', '226.63', '226.65', '226.68', '73.30384909', 58], [1733748300, '226.66', '226.69', '225.94', '225.94', '226.10', '95.68620769', 34], [1733748600, '226.00', '227.05', '225.79', '227.01', '226.70', '498.33596086', 115], [1733748900, '227.08', '227.60', '227.04', '227.60', '227.17', '253.30340842', 69], [1733749200, '227.54', '227.69', '226.77', '226.77', '227.52', '526.34030934', 64], [1733749500, '226.92', '227.46', '226.92', '227.29', '227.39', '568.33031128', 59], [1733749800, '227.26', '227.67', '227.26', '227.62', '227.57', '18

TypeError: object of type 'int' has no len()

In [37]:

# asset_symbol = "XBTUSD"  # Example asset symbol for Kraken (Bitcoin/USD)
start_date = "2023-11-01"
end_date = "2023-11-25"
total_capital = 10000  # Example capital
api_key = cfg.api_key
private_key = cfg.api_priv


# Step 1: Get Historical Data
historical_data = get_historical_data(assetPairs, start_date, end_date)

# Step 2: Calculate Entry and Exit Points
anticipated_pct_change = calculate_anticipated_pct_change(historical_data)
support_level, resistance_level = calculate_support_resistance(historical_data, anticipated_pct_change)

# Step 3: Risk Management
entry_price = historical_data['Close'].iloc[-1]
risk_reward_ratio = 3  # Example risk-reward ratio
stop_loss, take_profit = calculate_risk_management_levels(entry_price, risk_reward_ratio)
position_size = calculate_position_size(total_capital, entry_price, stop_loss)

# Plot the historical data with support and resistance levels for each symbol in assetPairs list
fig, axs = plt.subplots(len(assetPairs), figsize=(12, 6 * len(assetPairs)))
if len(assetPairs) == 1:
    axs = [axs]  # Ensure axs is iterable when there's only one subplot

for i, asset_symbol in enumerate(assetPairs):
    asset_data = historical_data[historical_data['AssetPair'] == asset_symbol]
    support_level, resistance_level = calculate_support_resistance(asset_data, anticipated_pct_change)
    
    axs[i].plot(asset_data['Close'], label='Close Price')
    axs[i].axhline(y=support_level, color='g', linestyle='--', label='Support Level')
    axs[i].axhline(y=resistance_level, color='r', linestyle='--', label='Resistance Level')
    axs[i].set_title(f"Price Chart for {asset_symbol}")
    axs[i].set_xlabel("Date")
    axs[i].set_ylabel("Price")
    axs[i].legend()

plt.tight_layout()
plt.show()


# Plot risk management levels
plt.figure(figsize=(12, 6))
plt.plot(historical_data['Close'], label='Close Price')
plt.axhline(y=stop_loss, color='r', linestyle='--', label='Stop Loss')
plt.axhline(y=take_profit, color='g', linestyle='--', label='Take Profit')
plt.title(f"Risk Management Levels for {asset_symbol}")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()

# Step 4: Execute the Trade
#trade_response = execute_trade(api_key, private_key, asset_symbol, position_size, entry_price)
#print("Trade executed successfully:", trade_response)   

KeyError: 'BTCUSD'